<a href="https://colab.research.google.com/github/s-vgustavo/python-ML-studies/blob/main/Web_Scraping_pt_br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Importando as bibliotecas que vou utilizar

In [ ]:
import requests
from zipfile import ZipFile
import os
import pandas as pd
print('Bibliotecas importadas.')

1. Baixando todos os arquivos

In [ ]:
#função para baixar arquivos
def baixar_arquivo(url, endereco):
  resposta = requests.get(url)
  if resposta.status_code == requests.codes.OK:
    with open (endereco, 'wb') as novo_arquivo:
      novo_arquivo.write(resposta.content)
    print(f'Download finalizado. Salvo em {endereco}.')

In [ ]:
#loop que baixa todos os arquivos
for i in range(2014, 2023):
  for j in range(1, 13):
    url = f'https://www.bcb.gov.br/content/estatisticas/rankingcambioinstituicoes/ESTATCAMBIF{i}{j:02d}-IF-{i}{j:02d}.zip'
    endereco = f'/content/drive/MyDrive/Edumi/case4/ranking{i}{j:02d}.zip'
    baixar_arquivo(url, endereco)

print('Arquivos baixados.')

2. Acessando os arquivos de interesse

In [ ]:
#extrair o conteudo dos zips
for i in range(2014, 2023):
  for j in range(1, 13):
    arquivo = f'/content/drive/MyDrive/Edumi/case4/ranking{i}{j:02d}.zip'
    if os.path.exists(arquivo) == True:
      with ZipFile(arquivo, 'r') as zipObj:
        # Extract all the contents of zip file in different directory
        zipObj.extractall(f'/content/drive/MyDrive/Edumi/case4/rankingfiles{i}{j:02d}')
        print(f'File is unzipped in rankingfiles{i}{j:02d} folder') 

print('Arquivos extraídos.')

3. Criando uma base de dados única

In [ ]:
#iterar sobre as pastas
#em cada pasta, abrir o arquivo específico usando concatenação
#transformar em dataframe e inserir um rastreador de data
#armazenar ele na memória
#juntar todos os dataframes em 1 só

pdList = []
pdList2 = []

for i in range(2014, 2023):
  for j in range(1, 13):
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles{i}{j:02d}/Ranking Instituiç╞o {i} {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='unicode_escape') as file:
        df = pd.read_csv(file, header = None, sep = ';', on_bad_lines='skip')
        df.insert(0, 'identificador', f'{i}{j:02d}')
        pdList.append(df)

print('DataFrame criado.')

for x in pdList:
  temp_df = pd.DataFrame(x) 
  temp_df = temp_df.iloc[7:]
  temp_df = temp_df.iloc[:-6]
  pdList2.append(temp_df)

print('Limpeza inicial do DataFrame concluída.')

df_final = pd.concat(pdList2, axis=0)
print('Base de dados criada e unificada com sucesso.')
print(df_final.head())

4. Limpando a base de dados

In [ ]:
#adicionando um cabeçalho
df_final.columns = ['identificador', 'rank', 'cod_instituicao', 'nome_instituicao', 'n_operacoes_export', 'valor_export', 'n_operacoes_import',
              'valor_import', 'n_oper_transf_do_ext', 'valor_transf_do_ext', 'n_oper_transf_para_ext', 'valor_transf_para_ext',
              'n_oper_mercado_primario', 'valor_mercado_primario', 'n_oper_interbank_compra', 'valor_interbank_compra', 
              'n_oper_interbank_venda', 'valor_interbank_venda', 'n_oper_total_interbank', 'valor_total_interbank',
              'n_oper_total', 'valor_total', 'data', 'unknown1', 'unknown2', 'unknown3']

In [ ]:
#resetando o index
df_final.reset_index(inplace=True)
df_final.head()

In [ ]:
#cast no identificador para transformá-lo em datetime (vai me ajudar no Power BI)
df_final['identificador'] = pd.to_datetime(df_final['identificador'], format='%Y%m')
df_final.head()

In [ ]:
#salvando o arquivo final para criar um checkpoint
print('Limpeza da base concluída com sucesso.')
df_final.to_csv(r'/content/drive/MyDrive/Edumi/df_final.csv', index=False, sep=';')
print('Progresso salvo na pasta Edumi do Google Drive.')

5. Criando visualizações para a dashboard

In [ ]:
#1 - 5 maiores instituições de câmcio em agosto de 2022
arq = '/content/drive/MyDrive/Edumi/df_final.csv'
with open(arq, 'r', encoding='unicode_escape') as file:
  file_df = pd.read_csv(file, sep=';')
  df_5maiores_agosto_2022 = file_df[['identificador', 'rank', 'nome_instituicao']]
  df_5maiores_agosto_2022 = df_5maiores_agosto_2022[(df_5maiores_agosto_2022['identificador'] > '2022-07-01')].head(5)
  df_5maiores_agosto_2022.reset_index(drop=True, inplace=True)
  print('Visualização das 5 maiores instituições de câmbio em agosto de 2022 criada.')
  df_5maiores_agosto_2022.to_csv('/content/drive/MyDrive/Edumi/novas/5maiores_agosto22.csv', index=False, sep=';')
  print(df_5maiores_agosto_2022)
  print('Visualização salva.')

Visualização das 5 maiores instituições de câmbio em agosto de 2022 criada.
  identificador rank               nome_instituicao
0    2022-08-01    1            ITAÃ UNIBANCO S.A.
1    2022-08-01    2            BANCO CITIBANK S.A.
2    2022-08-01    3         BANCO J.P. MORGAN S.A.
3    2022-08-01    4  BANCO SANTANDER (BRASIL) S.A.
4    2022-08-01    5           BANCO DO BRASIL S.A.
Visualização salva.


In [ ]:
#2 - As 5 maiores instituições de câmbio no Brasil no ano de 2022

arq = '/content/drive/MyDrive/Edumi/df_final.csv'
with open(arq, 'r', encoding='unicode_escape') as file:
  file_df = pd.read_csv(file, sep=';')
  df_top5_2022 = file_df[['identificador', 'rank', 'nome_instituicao']]
  df_top5_2022 = df_top5_2022[(df_top5_2022['identificador'] > '2022')]
  df_top5_2022['rank'] = df_top5_2022['rank'].apply(pd.to_numeric, errors='coerce')
  df_top5_2022 = df_top5_2022[['rank', 'nome_instituicao']].groupby('nome_instituicao').mean().sort_values(by='rank', ascending=True).head(5)
  print('Visualização das 5 maiores instituições de câmbio de 2022 criada.')
  df_top5_2022.to_csv('/content/drive/MyDrive/Edumi/novas/top5_2022.csv', index=False, sep=';')
  print(df_top5_2022)
  print('Visualização salva.')

Visualização das 5 maiores instituições de câmbio de 2022 criada.
                                rank
nome_instituicao                    
BANCO CITIBANK S.A.            1.875
BANCO SANTANDER (BRASIL) S.A.  2.375
ITAÃ UNIBANCO S.A.            2.500
BANCO J.P. MORGAN S.A.         3.500
BANCO DO BRASIL S.A.           5.750
Visualização salva.


In [ ]:
#3 A evolução das instituições nos últimos 12 meses

pdList = []

for j in range(1, 9): #pegando os de 2022
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles2022{j:02d}/Ranking Instituiç╞o 2022 {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='latin1') as file:
        df = pd.read_csv(file, sep=';', skiprows=6)
        df.insert(0, 'identificador', f'2022{j:02d}')
        df.drop(index=df.index[-1],axis=0,inplace=True)
        df.columns = df.columns.str.strip()
        pdList.append(df)

for k in range(8, 13): #pegando os de 2021
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles2021{j:02d}/Ranking Instituiç╞o 2021 {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='latin1') as file:
        df = pd.read_csv(file, sep=';', skiprows=6)
        df.insert(0, 'identificador', f'2021{j:02d}')
        df.drop(index=df.index[-1],axis=0,inplace=True)
        df.columns = df.columns.str.strip()
        pdList.append(df)

df12meses = pd.concat(pdList, axis=0)
df12meses.columns = ['identificador', 'rank', 'cnpj', 'nome_inst', 'export_quant', 'export_valor',
                    'import_quant', 'import_valor', 'transf_ext_quant', 'transf_ext_valor',
                    'transf_p_ext_quant', 'transf_p_ext_valor', 'total_primario_quant',
                     'total_primario_valor', 'interbanc_compra_quant',
                    'interbanc_compra_valor', 'interbanc_venda_quant', 'interbanc_venda_valor',
                    'total_interbanc_quant', 'total_interbanc_valor', 'total_quant', 'total_valor', 'unknown']

df12meses['rank'] = df12meses['rank'].apply(pd.to_numeric, errors='coerce')
df12meses['rank'].dropna(axis=0, how='any')
df12meses['identificador'] = pd.to_datetime(df12meses['identificador'], format='%Y%m')
df12meses_view = df12meses[['identificador', 'rank', 'nome_inst']].groupby('identificador', 'nome_inst')

#df12meses_view.to_csv('/content/drive/MyDrive/Edumi/visualizacoes case 4/rank12meses.csv', index=False)

print(df12meses.head())

In [ ]:
#4 Quantos contratos de Câmbio já foram fechados esse ano a cada mês
#fail
pdList = []

for j in range(1, 9):
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles2022{j:02d}/Ranking Instituiç╞o 2022 {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='latin1') as file:
        df = pd.read_csv(file, sep=';', skiprows=6)
        df.insert(0, 'identificador', f'2022{j:02d}')
        df.drop(index=df.index[-1],axis=0,inplace=True)
        df.columns = df.columns.str.strip()
        pdList.append(df)
  
dfcontratos2022 = pd.concat(pdList, axis=0)
dfcontratos2022.columns = ['identificador', 'rank', 'cnpj', 'nome_inst', 'export_quant', 'export_valor',
                    'import_quant', 'import_valor', 'transf_ext_quant', 'transf_ext_valor',
                    'transf_p_ext_quant', 'transf_p_ext_valor', 'total_primario_quant',
                     'total_primario_valor', 'interbanc_compra_quant',
                    'interbanc_compra_valor', 'interbanc_venda_quant', 'interbanc_venda_valor',
                    'total_interbanc_quant', 'total_interbanc_valor', 'total_quant', 'total_valor', 'unknown']

dfcontratos2022[['identificador', 'rank', 'total_quant']] = dfcontratos2022[['identificador', 'rank', 'total_quant']].apply(pd.to_numeric, errors='coerce')
dfcontratos2022[['identificador', 'rank', 'total_quant']].dropna(axis=0, how='any')
dfcontratos2022['identificador'] = pd.to_datetime(dfcontratos2022['identificador'], format='%Y%m')

dfcontratos2022['total_quant'].groupby(by = 'identificador')
#dfcontratos2022.head()

In [ ]:
#5 Qual das modalidades de Câmbio (Compra, Venda, Exportação, Importação) é a maior no ano atual? Essa modalidade é a maior desde quando?

pdList = []

for j in range(1, 9):
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles2022{j:02d}/Ranking Instituiç╞o 2022 {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='latin1') as file:
        df = pd.read_csv(file, sep=';', skiprows=6)
        df.insert(0, 'identificador', f'2022{j:02d}')
        df.drop(index=df.index[-1],axis=0,inplace=True)
        df.columns = df.columns.str.strip()
        pdList.append(df)
  
dfmodalidades = pd.concat(pdList, axis=0)
dfmodalidades.columns = ['identificador', 'rank', 'cnpj', 'nome_inst', 'export_quant', 'export_valor',
                    'import_quant', 'import_valor', 'transf_ext_quant', 'transf_ext_valor',
                    'transf_p_ext_quant', 'transf_p_ext_valor', 'total_primario_quant',
                     'total_primario_valor', 'interbanc_compra_quant',
                    'interbanc_compra_valor', 'interbanc_venda_quant', 'interbanc_venda_valor',
                    'total_interbanc_quant', 'total_interbanc_valor', 'total_quant', 'total_valor', 'unknown']

dfmodalidades[['transf_p_ext_quant']] = dfmodalidades[['transf_p_ext_quant']].apply(pd.to_numeric, errors='coerce')
dfmodalidades[['transf_p_ext_quant']].dropna(axis=0, how='any')

print(dfmodalidades[['export_quant', 'import_quant', 'transf_ext_quant', 'transf_p_ext_quant']].sum())

dfmodalidades[['export_quant', 'import_quant', 'transf_ext_quant', 'transf_p_ext_quant']].sum().to_csv('/content/drive/MyDrive/Edumi/visualizacoes case 4/modalidades12meses.csv')


export_quant           73519.440
import_quant          140283.336
transf_ext_quant      149585.078
transf_p_ext_quant    252804.937
dtype: float64


3. Concatenando os arquivos em um único grande arquivo

4. Criando um checkpoint para não perder o df original

In [ ]:
#salvando o df1 para ter um checkpoint do trabalho
df1.to_csv(r'/content/drive/MyDrive/Edumi.csv', index=False)

In [ ]:
#criando uma cópia do df para garantir que não vamos alterá-lo
df2 = df1.copy()
df2.head()

5. Análise exploratória

In [ ]:
#convertendo a coluna de identificador para datetime, assim eu vou poder filtrar por data
df2['identificador'] = pd.to_datetime(df['identificador'], format='%Y%m')
df2.dtypes

identificador                 datetime64[ns]
Rank                                  object
Código Instit.                        object
Nome da Instituição                   object
Exportação                            object
Unnamed: 4                            object
Importação                            object
Unnamed: 6                            object
Transferência do Exterior             object
Unnamed: 8                            object
Transferência p/ Exterior             object
Unnamed: 10                           object
Total do Primário                     object
Unnamed: 12                           object
Interbancário - Compra                object
Unnamed: 14                           object
Interbancário - Venda                 object
Unnamed: 16                           object
Total do Interbancário                object
Unnamed: 18                           object
Total                                 object
Unnamed: 20                           object
Unnamed: 2

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17165 entries, 0 to 164
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   identificador                   16003 non-null  datetime64[ns]
 1   Rank                            16606 non-null  object        
 2   C¢digo Instit.                  1393 non-null   object        
 3   Nome da InstituiÆo             1401 non-null   object        
 4   ExportaÆo                      750 non-null    object        
 5   Unnamed: 4                      10163 non-null  object        
 6   ImportaÆo                      915 non-null    object        
 7   Unnamed: 6                      11472 non-null  object        
 8   Transferncia do Exterior       1318 non-null   object        
 9   Unnamed: 8                      15698 non-null  object        
 10  Transferncia p/ Exterior       1390 non-null   object        
 11  Unna

In [ ]:
pdList = []

for i in range(2021, 2022):
  for j in range(1, 6):
    pasta_arquivo = f'/content/drive/MyDrive/Edumi/case4/rankingfiles{i}{j:02d}/Ranking Instituiç╞o {i} {j:02d}.csv'
    if os.path.exists(pasta_arquivo) == True:
      with open(pasta_arquivo, 'r', encoding='latin1') as file:
        df = pd.read_csv(file, sep=';', skiprows=5, on_bad_lines='skip')
        df.insert(0, 'identificador', f'{i}{j:02d}')
        df.columns = df.columns.str.strip()
        pdList.append(df)
  
df3 = pd.concat(pdList, axis=0)
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 826 entries, 0 to 160
Data columns (total 24 columns):
 #   Column                                                                                                                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                                                                                                                  --------------  -----  
 0   identificador                                                                                                                                                                                                                           826 non-null    object 
 1   Rank                                                                                                       

In [ ]:
#tratando 2021 05
arq = f'/content/drive/MyDrive/Edumi/case4/rankingfiles202105/Ranking Instituiç╞o 2021 05.csv'
with open(arq, 'r', encoding='latin1') as file:
  file_df = pd.read_csv(file, sep=';', skiprows=5)
  file_df.columns = file_df.columns.str.strip()
  file_df.pop('CNPJ da InstituiÃ§Ã£o')
  print(file_df.head())
  file_df.to_csv(arq, index=False)

In [ ]:
def check_row(row):
    if isinstance(row[0], int) == True:
        return True
    return False

    #skiprows= lambda x:check_row(x))

In [ ]:
#juntar dataframes

In [ ]:
def baixar_arquivo(url, endereco):
  resposta = requests.get(url)
  if resposta.status_code == requests.codes.OK:
    with open (endereco, 'wb') as novo_arquivo:
      novo_arquivo.write(resposta.content)
    print(f'Download finalizado. Salvo em {endereco}.')
  else:
    resposta.raise_for_status()


if __name__ == "__main__":
  BASE_URL = 'link'
  OUTPUT_DIR = 'output'

  for i in range(1, quantos?):
    nome_arquivo = os.path.join(OUTPUT_DIR, 'nome do arquivo{}.csv'.format(i))
    baixar_arquivo(BASE_URL.format(i), 'test.pdf')

In [ ]:
import pandas as pd

In [ ]:
with open('/content/temp/Ranking Instituiç╞o 2021 06.csv', 'r', encoding='latin1') as csv:
  df = pd.read_csv(csv, sep=',', encoding='utf-8')
  df.head()